## Importing neccessary libraries

In [1]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# Webscraping

In [7]:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating')
driver.implicitly_wait(15)

soup = BeautifulSoup(driver.page_source, 'html.parser')

# Getting all the movies
movies = soup.find_all('div', class_ = 'lister-item mode-advanced')

# Scraping first 5 pages for demostration
first_5_page = []

for page in range(0, 5):
    for movie in movies:
        # Getting title
        title = movie.h3.a.text.strip()
        
        # Getting the year
        year = movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.strip('()')
        
        # Getting certificate
        certificate = 0

        try:
            certificate = movie.p.find('span', class_ = 'certificate').text.strip()
        except:
            certificate = 'NA'

        # Getting runtime
        runtime = movie.p.find('span', class_ = 'runtime').text.strip(' mins')

        # Getting genre
        genre = movie.p.find('span', class_ = 'genre').text.strip()

        # Getting rating
        rating = movie.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')

        # Getting metascore
        # It is given that some might not have metascore
        metascore = movie.find('span', class_ = 'metascore').text.strip() if movie.find('span', class_ = 'metascore') else 'NA'
        
        # Getting description
        description = movie.find_all('p', class_ = 'text-muted')[1].text.strip()
        
        # Getting director(s)
        director_text =  movie.find('p', class_ = '').text.strip().replace('\n', '').split('|')[0]
        director = director_text[director_text.find(':') + 1:] # taking everything from colon onwards

        # Getting stars
        stars_text = movie.find('p', class_ = '').text.strip().replace('\n', '').split('|')[1]
        stars = stars_text[stars_text.find(':') + 1:] # taking everything from colon onwards
        
        # Getting votes
        votes = movie.find_all('span', attrs = {'name': 'nv'})[0].text if movie.find_all('span', attrs = {'name': 'nv'}) else 'NA'

        # Getting gross
        # Some movies do not show gross like 'It's a wonderful life'
        gross = movie.find_all('span', attrs = {'name': 'nv'})[1].text if len(movie.find_all('span', attrs = {'name': 'nv'})) > 1 else 'NA'

        row = {
            'title': title,
            'year': year,
            'certificate': certificate,
            'runtime': runtime,
            'genre': genre,
            'description': description,
            'rating': rating,
            'metascore': metascore,
            'director': director,
            'stars': stars,
            'votes': votes,
            'gross': gross    
        }

        first_5_page.append(row)

    if page == 0:
        driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[4]/a').click()
    
    else: 
        driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[4]/a[2]').click()

driver.close()

In [8]:
data = pd.DataFrame(first_5_page)

In [9]:
data.head()

,title,year,certificate,runtime,genre,description,rating,metascore,director,stars,votes,gross
0,The Shawshank Redemption,1994,15,142,Drama,Two imprisoned men bond over a number of years...,9.3,81,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","2,675,605",$28.34M
1,The Godfather,1972,X,175,"Crime, Drama",The aging patriarch of an organized crime dyna...,9.2,100,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","1,854,602",$134.97M
2,The Dark Knight,2008,12A,152,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,9.0,84,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","2,648,774",$534.86M
3,The Lord of the Rings: The Return of the King,2003,12A,201,"Action, Adventure, Drama",Gandalf and Aragorn lead the World of Men agai...,9.0,94,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...","1,843,835",$377.85M
4,Schindler's List,1993,15,195,"Biography, Drama, History","In German-occupied Poland during World War II,...",9.0,94,Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...","1,354,313",$96.90M
